In [ ]:
import os
import sys
import numpy as np
import sys
import matplotlib.pyplot as plt
from PIL import Image
import random
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

## Model Implementation

In [ ]:
import segmentation_models_pytorch as smp

model = smp.Unet(
    encoder_name="resnet18",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

## Loaders

#### Supervised Loader

In [ ]:
channel_means = [0.485, 0.456, 0.406]
channel_stds  = [0.229, 0.224, 0.225]
#################################################################################################

class MyDataset(Dataset):
    def __init__(self, image_paths, target_paths, train=True):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.train = train
        self.files = os.listdir(self.image_paths)
        self.labels = os.listdir(self.target_paths)
        
        self.color_jitter = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.1)

    def transform(self, image, mask):
        # Resize
        resize = transforms.Resize(size=(256, 256))
        image = resize(image)
        mask = resize(mask)
        
        # Random transformations only for training
        if self.train:
            # Random horizontal flipping
            if random.random() > 0.5:
                image = TF.hflip(image)
                mask = TF.hflip(mask)

            # Random vertical flipping
            if random.random() > 0.5:
                image = TF.vflip(image)
                mask = TF.vflip(mask)

        # Transform to tensor
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)
        
        # Normalize
        image = TF.normalize(image, channel_means, channel_stds)
        
        return image, mask
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = self.files[idx]
        label_name = self.labels[idx]
        image = Image.open(os.path.join(self.image_paths, img_name))

        # Color jitter only for training
        if self.train:
            image = self.color_jitter(image)
        
        mask = Image.open(os.path.join(self.target_paths, label_name)).convert("L")
        x, y = self.transform(image, mask)
        return x, y, img_name, label_name

In [ ]:
DIR_IMG_tr  = os.path.join("./train/"
                        , 'images')
DIR_MASK_tr = os.path.join("./train/"
                        , 'masks')
DIR_IMG_v  = os.path.join("./validation/"
                        , 'images')
DIR_MASK_v = os.path.join("./validation/"
                        , 'masks')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_T = MyDataset(DIR_IMG_tr, DIR_MASK_tr)
dataset_V = MyDataset(DIR_IMG_v, DIR_MASK_v, train=False)
#Batch Size and Loaders
batch_size = 4
train_loader = DataLoader(dataset_T, batch_size, shuffle=True, 
                          pin_memory=torch.cuda.is_available())
valid_loader = DataLoader(dataset_V, batch_size, shuffle=False, 
                          pin_memory= torch.cuda.is_available())

In [ ]:
for i, (input, target, img_name, label_name) in enumerate(valid_loader):
    
    #To Device
    input_var  = input.cuda()
    target_var = torch.round(target.cuda())
    print(input_var.shape, target_var.shape)
    
    plt.imshow(input_var[0].cpu().permute(1, 2, 0))
    plt.show()
    plt.imshow(target_var[0][0].cpu())
    plt.show()
    #print(torch.unique(target))
    break

#### Unsupervised Loader

In [ ]:
class MyDataset_UnSupervised(Dataset):
    def __init__(self, image_paths, train=True):
        self.image_paths = image_paths
        self.files = os.listdir(self.image_paths)

    def transform(self, image):  
        # Resize
        resize = transforms.Resize(size=(256, 256))
        image = resize(image)
        
        # Random horizontal flipping
        if random.random() > 0.2:
            image = TF.hflip(image)    
        # Random vertical flipping
        if random.random() > 0.2:
            image = TF.vflip(image)  
            
        # Random vertical flipping
        image2 = image.copy()
        
        t_c = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
        t_g = transforms.GaussianBlur(5, sigma=(0.1, 2.0))
        
        image2 = t_c(image2) 
        
        if random.random() < 0.3:
            image2 = t_g(image2) 
             
        # Transform to tensor
        image = TF.to_tensor(image) 
        image2 = TF.to_tensor(image2) 
        
        #Normalise
        image = TF.normalize(image, channel_means, channel_stds)
        image2 = TF.normalize(image2, channel_means, channel_stds)
        return image, image2
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self,idx):
        img_name = self.files[idx]
        image = Image.open(os.path.join(self.image_paths,img_name))
        image, image2 = self.transform(image)
        return image, image2, img_name

In [ ]:
DIR_IMG  = os.path.join("./unlabelled/"
                        , 'images')

dataset_Unsupervised = MyDataset_UnSupervised(DIR_IMG)

#Batch Size and Loaders
batch_size_unsupervised = 8
unsupervised_loader = DataLoader(dataset_Unsupervised, batch_size_unsupervised, shuffle = True, 
                          pin_memory=torch.cuda.is_available())#torch.cuda.is_available())
print(len(dataset_Unsupervised)/batch_size_unsupervised)

In [ ]:
import matplotlib.pyplot as plt
for i, (image, image2, img_name) in enumerate(unsupervised_loader):
    
    plt.imshow(image[0].permute(1, 2, 0))
    plt.show()
    plt.imshow(image2[0].permute(1, 2, 0))
    plt.show()

    print("Input", input_var.shape, img_name)
    break

#### Test Loader

In [ ]:
DIR_IMG_T  = os.path.join("./Test/", 'images')
DIR_MASK_T = os.path.join("./Test/", 'masks')
dataset_T = MyDataset(DIR_IMG_T, DIR_MASK_T, train = False)

#Batch Size and Loaders
batch_size_T = batch_size
test_loader = DataLoader(dataset_T, batch_size_T, shuffle=False, pin_memory=torch.cuda.is_available())#torch.cuda.is_available())

## Training and Validation

#### Losses

In [ ]:
from Loss import IoULoss
seg_loss = IoULoss()
CEloss = torch.nn.BCELoss()

#### Training Def

In [ ]:
import copy
import scipy.ndimage as ndimage

def train(train_loader, model,
          optimizer_, validation,
          MVL = 10, scheduler_=None):

    TrEpochsloss_Su = []
    TrEpochsloss_Unsu = []
    ValEpochsloss = []
    
    #Initial min_val_loss
    min_val_los = MVL

    for epoch in range(start_epoch, end_epoch):
            
        Supervisedloss = []    
        Unsupervisedloss = []
        
        model.train()
        ###################################################################################################
        dataloader_iterator = iter(train_loader)  ########################### S dataset
        for i, (image1, image2, image_name) in enumerate(unsupervised_loader): ######## L dataset
            ## S
            try:
                input, target, img_name, label_name = next(dataloader_iterator)
            except StopIteration:
                dataloader_iterator = iter(train_loader)    ################## S dataset
                input, target, img_name, label_name = next(dataloader_iterator)
            ##--------------------------------------------------
            #Target and Input
            ##--------------------------------------------------
            target = torch.round(target)
            #---------------------------------------------------
            ## Supervised
            #---------------------------------------------------
            #Model the input
            masks_su = model(input.cuda())
            masks_su = torch.sigmoid(masks_su)
            #Calcualte the Loss
            loss_Su = seg_loss(masks_su, target.cuda().squeeze(1).long())
            #Step loss appending
            Supervisedloss.append(loss_Su.item())
            
            #---------------------------------------------------
            ## Semi-Supervised
            #---------------------------------------------------
            #Model the input
            ###############################################Model
            masks_unsu1 = model(image1.cuda())
            masks_unsu1 = torch.sigmoid(masks_unsu1)
                                        
            masks_unsu2 = model(image2.cuda())
            masks_unsu2 = torch.sigmoid(masks_unsu2)
                                        
            #Calcualte the Loss
            loss_Unsu = seg_loss(masks_unsu2, torch.round(masks_unsu1).detach().float().cuda().squeeze(1))

            #Step loss appending
            Unsupervisedloss.append(loss_Unsu.item())
            
            beta = 1
            ### Losesse (Sup + UNSup)
            Loss = (loss_Su + beta * loss_Unsu)/(1+beta)
            
            # compute gradient and do SGD step            
            Loss.backward()

            #updating parameters of the model
            #Teacher1
            optimizer_.step()
            optimizer_.zero_grad()

            sys.stdout.write("\r Training %d/%d |M Su: %.3f Unsu: %.3f|"\
                 % (i, len(unsupervised_loader.dataset)/unsupervised_loader.batch_size, 
                    loss_Su.item(), loss_Unsu.item()))
            sys.stdout.flush()
          
        #Validation for one epoch
        valid_metrics = validation(model, valid_loader)
        valid_loss = valid_metrics['valid_loss']
        ###################################################################################
        
        #Saving Epochs Loss for Train and Validation
        #model     
        TrEpochsloss_Su.append(np.mean(Supervisedloss))
        TrEpochsloss_Unsu.append(np.mean(Unsupervisedloss))
        ValEpochsloss.append(valid_loss)
        
        ##PRITNIGN ONE EPOCH OUTCOME
        print("Epoch:", epoch, "###########################################################")
        print(f'\tTrain_loss = {np.mean(Supervisedloss):.5f}')
        print(f'\tTrain_loss = {np.mean(Unsupervisedloss):.5f}')
        print(f'\tValid_loss = {valid_loss:.5f}')
        print("LR at the end of epoch=", get_lr(optimizer_))
        
        #Save the model of the current epoch
        if valid_loss < min_val_los:
            print("valid_loss < min_val_loss")
            min_val_los = valid_loss
            best_wts = copy.deepcopy(model.state_dict())
            EPOCH_best = epoch
        
        #tune the learning rate        
        if scheduler_ is not None:
            scheduler_.step()
        print()
        
    #Best model based on validation
    model.load_state_dict(best_wts)
    
    return model, TrEpochsloss_Su, TrEpochsloss_Unsu, ValEpochsloss, EPOCH_best

def validate(model, val_loader):
    losses = AverageMeter()
    model.train(False)
    with torch.no_grad():

        for i, (input, target, fname, mname) in enumerate(val_loader):
            input_var  = input.cuda()
            target_var = torch.round(target).cuda()

            masks_pred = model(input_var)
            masks_pred = torch.sigmoid(masks_pred)
            
            loss = seg_loss(masks_pred, target_var.squeeze(1).long())

            losses.update(loss.item(), input_var.size(0))

    return {'valid_loss': losses.avg}

def save_check_point(state, is_best, file_name = 'checkpoint.pth.tar'):
    torch.save(state, file_name)
    if is_best:
        shutil.copy(file_name, 'model_best.pth.tar')

def calc_crack_pixel_weight(mask_dir):
    avg_w = 0.0
    n_files = 0
    for path in Path(mask_dir).glob('*.*'):
        n_files += 1
        m = ndimage.imread(path)
        ncrack = np.sum((m > 0)[:])
        w = float(ncrack)/(m.shape[0]*m.shape[1])
        avg_w = avg_w + (1-w)

    avg_w /= float(n_files)

    return avg_w / (1.0 - avg_w)

##@@@#@#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
##@@@#@#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [ ]:
model = model.cuda()

In [ ]:
##Epochs
start_epoch = 0 # start from start_epoch
end_epoch = 51 # the last epoch is end_epoch-1

optimizer1 = torch.optim.SGD(model.parameters(), lr = 0.0005, momentum=0.9, weight_decay=1e-3)
scheduler1 = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=5, gamma=0.9)

In [ ]:
model.load_state_dict(torch.load("Sup_St1.pth"))

In [ ]:
validate(model, valid_loader)

In [ ]:
model, TrEpochsloss_M1_Su, TrEpochsloss_M1_Unsu, ValEpochsloss_M1, EPOCH_best_M1 = \
train(train_loader, model,
          optimizer1, validate,
          MVL = 1, scheduler_ = scheduler1)

In [ ]:
validate(model, valid_loader)

In [ ]:
torch.save(model.state_dict(), "Semisup.pth")

#### Testing

In [ ]:
class MyDataset_A(Dataset):
    def __init__(self, image_paths, target_paths, train=False):
        self.image_paths = image_paths
        self.target_paths = target_paths
        self.files = os.listdir(self.image_paths)
        self.lables = os.listdir(self.target_paths)
        
    def transform(self, image, mask):
        # Resize
        resize = transforms.Resize(size=(256, 256))
        image = resize(image)
        mask = resize(mask)
        # Transform to tensor
        image = TF.to_tensor(image)
        mask = TF.to_tensor(mask)
        
        #Normalise
        image = TF.normalize(image, channel_means, channel_stds)
        
        return image, mask
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self,idx):
        img_name = self.files[idx]
        label_name = self.lables[idx]
        image = Image.open(os.path.join(self.image_paths,img_name))
        mask = Image.open(os.path.join(self.target_paths,label_name)).convert("L")
        x, y = self.transform(image, mask)
        return x, y, img_name, label_name

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def dice(y_true, y_pred):
    return (2 * (y_true * y_pred).sum() + 1e-15) / (y_true.sum() + y_pred.sum() + 1e-15)

def jaccard(y_true, y_pred):
    intersection = (y_true * y_pred).sum()
    union = y_true.sum() + y_pred.sum() - intersection
    return (intersection + 1e-15) / (union + 1e-15)

In [ ]:
# Test data path
DIR_IMG_test  = os.path.join("./test/", 
                             'images')
DIR_MASK_test = os.path.join("./test/", 
                             'masks')
dataset_test = MyDataset_A(DIR_IMG_test, DIR_MASK_test)

#Batch Size and Loader
batch_size = 1
test_loader_A = DataLoader(dataset_test, batch_size, shuffle=False, 
                          pin_memory=torch.cuda.is_available())
print(len(test_loader_A.dataset))

DICE = []
MIOU = []
k = 0

# running the test loop   
with torch.no_grad():

    model.eval()
    for i, (input, target, fname, MNAME) in enumerate(test_loader_A):

        #To Device
        input_var  = input.cuda()
        target_var = torch.round(target).cuda()
        k = k + 1
        #Torch Autocast
        with autocast():

            #Model the input
            masks_pred = model(input_var)
            ## Probabiliteis
            masks_pred = F.sigmoid(masks_pred)
            #round the probabiliteis to zeros and ones
            masks_pred = torch.round(masks_pred)

            #Making sure they are both tensors
            target_var = torch.tensor(target_var)
            masks_pred = torch.tensor(masks_pred)

            #calcualting Dice and IoU
            dice_score = dice(masks_pred, target_var[0])
            jaccard_score = jaccard(masks_pred, target_var[0])

            #Appending the calculated metrics
            DICE.append(dice_score.item())
            MIOU.append(jaccard_score.item())

print("Dice", np.mean(np.array(DICE)), np.std(np.array(DICE)))
print("IoU", np.mean(np.array(MIOU)), np.std(np.array(MIOU)))